In [7]:
import abc
import collections
import enum
import math
import pathlib
import typing
import warnings

import numpy as np
import torch
import torch.optim
import torch.utils.data
import tqdm
from matplotlib import pyplot as plt
from solution import *

In [8]:
data_dir = pathlib.Path.cwd()
model_dir = pathlib.Path.cwd()
output_dir = pathlib.Path.cwd()

# Load training data
train_xs = torch.from_numpy(np.load(data_dir / "train_xs.npz")["train_xs"])
raw_train_meta = np.load(data_dir / "train_ys.npz")
train_ys = torch.from_numpy(raw_train_meta["train_ys"])
train_is_snow = torch.from_numpy(raw_train_meta["train_is_snow"])
train_is_cloud = torch.from_numpy(raw_train_meta["train_is_cloud"])
dataset_train = torch.utils.data.TensorDataset(train_xs, train_is_snow, train_is_cloud, train_ys)

# Load validation data
val_xs = torch.from_numpy(np.load(data_dir / "val_xs.npz")["val_xs"])
raw_val_meta = np.load(data_dir / "val_ys.npz")
val_ys = torch.from_numpy(raw_val_meta["val_ys"])
val_is_snow = torch.from_numpy(raw_val_meta["val_is_snow"])
val_is_cloud = torch.from_numpy(raw_val_meta["val_is_cloud"])
dataset_val = torch.utils.data.TensorDataset(val_xs, val_is_snow, val_is_cloud, val_ys)

# Fix all randomness
setup_seeds()

# Build and run the actual solution
train_loader = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=16,
    shuffle=True,
    num_workers=0,
)

swag = SWAGInference(
    train_xs=dataset_train.tensors[0],
    model_dir=model_dir,
)

In [9]:
swag.fit(train_loader)
swag.calibrate(dataset_val)

with torch.random.fork_rng():
    evaluate(swag, dataset_val, EXTENDED_EVALUATION, output_dir)

Loaded pretrained MAP weights from c:\Users\Jeremias\Documents\ETH\Semester7\PAI\task2\map_weights.pt


Running gradient descent for SWA:  53%|█████▎    | 16/30 [08:31<07:27, 31.95s/it, lr=0.045, avg. epoch loss=0.462, avg. epoch accuracy=0.837]


KeyboardInterrupt: 

AttributeError: module 'numpy' has no attribute 'softmax'